1. Перевод в ч/б
2. Перевод в градации серого
3. Повышение резкости
4. Повышение контрастности
5. Баланс белого
6. Инверсия

In [1]:
filename = 'sample.jpg'
filepath = './static/uploads/sample.jpg'

In [2]:
from PIL import Image
import cv2
import numpy as np

In [3]:
img_original = Image.open(filepath)

In [4]:
def white_balance_loops(img):
    result = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    for x in range(result.shape[0]):
        for y in range(result.shape[1]):
            l, a, b = result[x, y, :]
            # fix for CV correction
            l *= 100 / 255.0
            result[x, y, 1] = a - ((avg_a - 128) * (l / 100.0) * 1.1)
            result[x, y, 2] = b - ((avg_b - 128) * (l / 100.0) * 1.1)
    result = cv2.cvtColor(result, cv2.COLOR_LAB2RGB)
    
    return result

In [6]:
images = {}
# Original 
images['orig'] = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
# Grayscale
images['gray'] = cv2.cvtColor(images['orig'], cv2.COLOR_RGB2GRAY)
# Black-white
_, images['bw'] = cv2.threshold(images['gray'], 127, 255, cv2.THRESH_BINARY)
# Sharpness
kernel_sharp = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
images['sharp'] = cv2.filter2D(images['orig'], -1, kernel_sharp)
# Contrast
alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)
images['cont'] = cv2.convertScaleAbs(images['orig'], alpha=alpha, beta=beta)
# White Balance
images['white'] = white_balance_loops(images['orig'])
# Inversion
images['inv'] = cv2.bitwise_not(images['orig'])

In [ ]:
list_of_images = []
for i_key, i_val in images.items():
    save_path = filepath[:-len(filename)] + i_key + "_" + filename
    list_of_images.append(save_path)
    Image.fromarray(i_val).save(save_path)

In [ ]:
list_of_images